# Notebook to test the Cormack algorithm for the inverse problem

## Problem statement:
$f(p, \phi) = \int_{L(\mathbf{p})} \,g(r, \theta)$, find $g(r, \theta)$ from $f(p, \phi)$.

In [ ]:
from utils import *
from math import factorial as fact
from scipy.special import jn, jn_zeros, jv, jvp # jvp is the derivative of jn
from scipy.integrate import quad

In [2]:
IDX = np.random.randint(0, 69352)
M = 2
L = 7
FINESSE = 110

RFX_I = 0 # index of the rfx fan [0:vdi, 1:vdc, 2:vde, 3:hor]

## Data

In [ ]:
#load data_clean.npy
d = np.load('data/data_clean.npy')
print(f'Keys in data: {d.dtype.names}')
SXR = d['data'][IDX][RFX_COMBINED_INTERVALS]
EMISS = d['emiss'][IDX]
BESSEL_COEFSS = d['target'][IDX]
# XX_GT, YY_GT = np.meshgrid(d['x_emiss'][IDX], d['y_emiss'][IDX][::-1]) # # NOTE: it's flipped up/down bc it's saved as an image with (0,0) at top left)
XX_GT, YY_GT = np.meshgrid(d['x_emiss'][IDX], d['y_emiss'][IDX]) # # NOTE: NOT FLIPPED
XY_GT = np.stack([XX_GT, YY_GT], axis=-1)
MAJR, MINR = d['majr'][IDX], d['minr'][IDX]
print(f'SXR: {SXR.shape}, emiss: {EMISS.shape}, BESSEL_COEFSS: {BESSEL_COEFSS.shape}, XY_GT: {XY_GT.shape}')

Bessel coeffs are 21, 3x7, because there are 2 coefss for cosine (0 and 1), both non-zero, and 1 for sine (1), bc the sine(0) is zero.

In [4]:
# a0cl, a1cl, a1sl = BESSEL_COEFSS[0:L], BESSEL_COEFSS[L:2*L], BESSEL_COEFSS[2*L:3*L]
A0C, A1C, A1S = BESSEL_COEFSS[0:L], BESSEL_COEFSS[L:2*L], BESSEL_COEFSS[2*L:3*L] 
A0S = np.zeros(L) #A0S = 0
A0, A1 = A0C + A0S *1j, A1C + A1S *1j # A0, A1 are complex numbers (see note below)
A = np.array([A0, A1], dtype=complex) # a is a MxL array of complex numbers

## Bessel functions and zeros

In [ ]:
#bessel
def Bessel(m:int,x:np.ndarray): # bessel function of the first kind
    s = x.shape
    return jv(m,x.reshape(-1)).reshape(s)
J = Bessel # J is the bessel function of the first kind

def Bessel_Zeros(m:int,n:int): # zeros of the m-th bessel function
    if m == 0: return jn_zeros(m,n)
    else: return np.concatenate([[0], jn_zeros(m,n-1)])
J0 = Bessel_Zeros # J0 is the zeros of the bessel function of the first kind
JZ = np.array([J0(m,L) for m in range(M)]) # J0ML is the zeros of the bessel function of the first kind

def Bessel_Derivative(m:int,x:np.ndarray): # derivative of the bessel function of the first kind
    return jvp(m,x)
JP = Bessel_Derivative # JP is the derivative of the bessel function of the first kind

def Tchebychev1(m:int,x:np.ndarray): # tchebyshev polynomial of the first kind
    return np.cos(m*np.arccos(x))
Tm = Tchebychev1 # T is the tchebyshev polynomial of the first kind

def Tchebychev2(m:int,x:np.ndarray): # tchebyshev polynomial of second kind
    return np.sin((m+1)*np.arccos(x))/np.sqrt(1-x**2)
Um = Tchebychev2 # U is the tchebyshev polynomial of the second kind

def Rml(m, l, r): # radial function
    return J(m, r*JZ[m,l])
    # return J(m, r*J0(m, L)[l])


# def Bessel(m:int,x:np.ndarray,n=70): # accurate until x ~ n+m
#     s = x.shape
#     x = x.reshape(-1)
#     b = np.zeros_like(x)
#     for l in range(n):
#         b += (-1)**l * (x/2)**(2*l+m) / (fact(l)*fact(l+m))
#     return b.reshape(s)

#plot
tmpM = 5
r = np.linspace(0, 30, 300)
plt.figure(figsize=(15,12))
plt.subplot(321)
for m in range(tmpM):
    plt.plot(r, J(m,r), label=f'm={m}')
plt.ylim(-1,1)
plt.title('Basil functions')

# plot the first bessel functions
plt.subplot(322)
for m in range(5):
    plt.plot(r, JP(m,r), label=f'm={m}')
plt.ylim(-1,1)
plt.title('Derivative Basil functions')

r = np.linspace(0, 1, 300)
for m in range(tmpM):
    plt.subplot(3, tmpM, tmpM+1+m)
    # for l in range(L):
    #     plt.plot(r, Rml(m, l, r), label=f'l={l}')
    plt.title(f'm={m}')
    plt.ylim(-1,1)
    plt.title(f'R_{m} = J({m}, r*J0({m}, {L}))')

# plot Tchebyshev polynomials
plt.subplot(325)
r = np.linspace(-1, 1, 300)
for m in range(tmpM):
    plt.plot(r, Tm(m,r), label=f'm={m}')
plt.title('Chebi\'s polynomials 1st kind')
plt.legend()

# plot Tchebyshev polynomials
plt.subplot(326)
r = np.linspace(-1, 1, 300)
for m in range(tmpM):
    plt.plot(r, Um(m,r), label=f'm={m}')
plt.title('Chebi\'s polynomials 2nd kind')
plt.legend()


plt.show()

In [ ]:
plt.figure(figsize=(20,6))
r = np.linspace(0, 1, 300)
for m in range(M):
    for l in range(L):
        plt.subplot(M, L, m*L+l+1)
        plt.plot(r, Rml(m, l, r), label=f'm={m}, l={l}')
        plt.title(f'm={m}, l={l}')
        plt.ylim(-1,1)
plt.suptitle('Radial functions (Bessel)')
plt.show()


## Reconstructing the emiss $g(r,\theta)$ from the Bessel coefficients
$g^{c,s}_m(r) = \sum_{l=0}^{\infty} a^{c,s}_{ml} J_{m}(x_{ml}r)$ , with $x_{ml}$ the $l$-th root of the $m$-th Bessel function.

In complex form:
$g_m(r) = \sum_{l=0}^{\infty} a_{ml} J_{m}(x_{ml}r)$

$r, \theta$ are the spherical coordinates, $m$ is the harmonic, $c,s$ are the cosine and sine coefficients, $a$ are the Bessel coefficients, $J_m$ is the Bessel function of the first kind of order $m$.

Note on imaginary/aritmetic version of the Fourier series:
- $\sum_{m=-\infty}^{\infty} a_m e^{im\theta} = \sum_{m=-\infty}^{\infty} a^c_m \cos(m\theta) + i a^s_m \sin(m\theta)$
- $a^c_m = a_m + a_{-m} = 2 Re(a_m)$ 
- $a^s_m = i(a_m - a_{-m}) = 2 Im(a_m)$

In [ ]:
def calc_g(r, θ, a): # r: vector of radii, θ: vector of angles, a vector of coefficients (complex numbers)
    assert a.shape == (M,L) and a.dtype==complex, f'a.shape: {a.shape}, a.dtype: {a.dtype}'
    g = np.zeros((len(r), len(θ)), dtype=complex)
    for m in range(M):
        gm = np.dot(a[m], J(m, r*JZ[m,:][:,None]))
        # gm = np.dot(a[m], J(m, r*J0(m, L)[:,None]))
        g += gm[:,None] * np.exp(1j*m*θ)
    return np.abs(g).T

def calc_g(r, θ, a): # equivalent (slower, but maybe clearer) implementation
    g = np.zeros((len(r), len(θ)), dtype=complex)
    for m in range(M):
        gm = np.zeros((len(r), L), dtype=complex)
        for l in range(L):
            gm[:,l] = a[m,l] * Rml(m, l, r)
        gm = np.sum(gm, axis=1)
        g += gm[:,None] * np.exp(1j*m*θ)
    return np.abs(g).T

r = np.linspace(0,1,FINESSE) # create a vector of radii
θ = np.linspace(0,2*π,FINESSE) # create a vector of angles

g = calc_g(r, θ, A)/MINR # calculate the g emissivity map

rr, θθ = np.meshgrid(r, θ) # create a meshgrid of r and θ
rθ = np.stack([rr, θθ], axis=-1)

xy = np.zeros_like(rθ) # convert to cartesian coordinates
xy[:,:,0] = rθ[:,:,0]*MINR * np.cos(rθ[:,:,1])
xy[:,:,1] = rθ[:,:,0]*MINR * np.sin(rθ[:,:,1]) 

#plot
plt.figure(figsize=(9,2))
cbar_min, cbar_max = np.min([np.min(g), np.min(EMISS)]), np.max([np.max(g), np.max(EMISS)])
xlm, xlM, ylm, ylM = np.min(XY_GT[:,:,0]), np.max(XY_GT[:,:,0]), np.min(XY_GT[:,:,1]), np.max(XY_GT[:,:,1])
plt.subplot(121)
plt.contourf(xy[:,:,0]+MAJR, xy[:,:,1], g)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.grid(False)
plt.xlim(xlm, xlM); plt.ylim(ylm, ylM)
plt.colorbar()
plt.subplot(122)
plt.contourf(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.grid(False)
plt.xlim(xlm, xlM); plt.ylim(ylm, ylM)
plt.colorbar()
plt.show()

## Getting the Bessel coefficients

### Line of signt definition and visualization
<img src="data/geom.jpg" alt="Geometry Image" width="300"/>

In [ ]:
#functions to convert line of sight in format (pinhole, angle) to (Φ, p) 
def phα2line(ph, α): # convert line of sight in format (pinhole, angle) to standard line in the form ax + by + c = 0, returns a, b, c
    return np.array([-np.sin(α), np.cos(α), np.sin(α)*ph[0]-np.cos(α)*ph[1]])
    
def line2pΦ(line): # convert standard line in the form ax + by + c = 0 to (Φ, p), returns Φ, p
    a, b, c = line
    #calc point on the line closeesto to (RM, ZM)
    xp = (b*(b*RM - a*ZM) - a*c)/(a**2 + b**2)
    yp = (a*(-b*RM + a*ZM) - b*c)/(a**2 + b**2)
    # return dist2line([RM, ZM], line), np.arctan2(-b, a)
    return np.sqrt((xp-RM)**2 + (yp-ZM)**2), np.arctan2(yp-ZM, xp-RM)

def dist2line(p, line): # calculate the distance from a point to a line, returns distance line: [a, b, c] p: [x, y]
    (x,y), (a,b,c) = p, line
    return np.abs(a*x + b*y + c)/np.sqrt(a**2 + b**2)



def draw_line(line, color='w', alpha=0.5, lw=1.5): # line: [a,b,c], ax+by+c=0,
    # assume to already have a plot
    a,b,c = line
    if np.abs(a) < np.abs(b):
        x = np.linspace(R0, R1, 200)
        y = -(a*x + c)/b
    else:
        y = np.linspace(Z0, Z1, 200)
        x = -(b*y + c)/a
    outside = ((x-RM)**2 + (y-ZM)**2) < (R_FW+GSPAC/2)**2
    x, y = x[outside], y[outside]
    plt.plot(x, y, alpha=alpha, color=color, lw=lw)

def intersect(l1, l2): # calculate the intersection of two lines, returns intersection point
    (a1, b1, c1), (a2, b2, c2) = l1, l2
    den = b1*a2 - a1*b2
    # assert np.abs(den) > 1e-6, f'Parallel lines'#: {l1}, {l2}, den: {den:.3f}, b1*a2: {b1*a2:.3f}, a1*b2: {a1*b2:.3f}'
    if np.abs(den) < 1e-6: return np.array([np.nan, np.nan]) # parallel lines
    x = (c1*b2 - b1*c2)/den
    y = (a1*c2 - a2*c1)/den
    return np.array([x, y])



# test 
nrays, start_angle, span_angle, pinhole_position, idxs_to_keep = RFX_SXR_NRAYS[RFX_I], RFX_SXR_STARTS[RFX_I], RFX_SXR_SPANS[RFX_I], RFX_SXR_PINHOLES[RFX_I], RFX_SXR_TO_KEEP[RFX_I]
rays, _, _ = create_rfx_fan(nrays, start_angle, span_angle, pinhole_position, idxs_to_keep, ret_all=True)

# calc angles
αs = np.linspace(start_angle, start_angle+span_angle, nrays)[idxs_to_keep]

lines = np.zeros((len(rays),3))
for i in range(len(rays)):
    lines[i] = phα2line(pinhole_position, αs[i])  

ps, Φs = np.zeros((len(rays)), dtype=float), np.zeros((len(rays)), dtype=float)
for i in range(len(rays)):
    ps[i], Φs[i] = line2pΦ(lines[i])

# plot
plt.figure(figsize=(9,2))
plt.subplot(121)
plt.contourf(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.xlim(R0, R1); plt.ylim(Z0, Z1)
plt.grid(False)
# plot rays
for r in rays: 
    plt.plot(r[:,0], r[:,1], 'w', alpha=0.8)

plt.subplot(122)
plt.contourf(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
# plt.xlim(R0, R1); plt.ylim(Z0, Z1)
plt.grid(False)
# plot lines
for l in lines: draw_line(l)

### LOS Conversion

In [ ]:
  # get rfx los
ps, Φs = np.zeros(len(SXR)), np.zeros(len(SXR))
lines = np.zeros((len(SXR),3))

idx = 0
for nr, startα, spanα, ph, keep_idxs in zip(RFX_SXR_NRAYS, RFX_SXR_STARTS, RFX_SXR_SPANS, RFX_SXR_PINHOLES, RFX_SXR_TO_KEEP):
    αs = np.linspace(startα, startα+spanα, nr)[keep_idxs]
    idx0 = idx
    for i in range(len(αs)):
        lines[idx] = phα2line(ph, αs[i])
        ps[idx], Φs[idx] = line2pΦ(lines[idx])
        idx += 1

# print(f'lines: \n{lines}')
# print(f'ps: {ps}')
# print(f'Φs: {Φs}')

# plot
plt.figure(figsize=(12,16))
plt.subplot(321)
plt.contourf(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.xlim(R0, R1); plt.ylim(Z0, Z1)
plt.grid(False)
for l in lines: draw_line(l, lw=1, alpha=.5)    

# now plot the lines, but the color is decided by the SXR value
plt.subplot(322)
# plt.contourf(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.xlim(R0, R1); plt.ylim(Z0, Z1)
# plt.grid(False)
# plot lines
for i, l in enumerate(lines): 
    draw_line(l, color=CMAP(SXR[i]/max(SXR)), lw=2, alpha=1)

# plot the intersection points
plt.subplot(323)
plt.contour(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.xlim(R0, R1); plt.ylim(Z0, Z1)
# plt.grid(False)
for l in lines: draw_line(l, lw=0.5, alpha=1)   

intersections, values = [], []
for i in range(0, len(lines)-(HOR1_INTERVAL[1]-HOR1_INTERVAL[0])): # do not intersect vertical los with other vertical los
    for j in range(len(lines)-(HOR1_INTERVAL[1]-HOR1_INTERVAL[0]), len(lines)):
        assert i != j, f'Intersection of the same line: {i}, {j}'
        try: p = intersect(lines[i], lines[j])
        except AssertionError as e: print(e)
        if ((p[0]-RM)**2 + (p[1]-ZM)**2) < (R_FW+GSPAC/2)**2:
            intersections.append(p)
            values.append((SXR[i] + SXR[j])/2)

intersections = np.array(intersections)
plt.plot(intersections[:,0], intersections[:,1], 'ro', ms=1.5)

plt.subplot(324)
plt.contour(XY_GT[:,:,0], XY_GT[:,:,1], EMISS)
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.axis('equal')
plt.xlim(R0, R1); plt.ylim(Z0, Z1)
# plt.grid(False)
plt.scatter(intersections[:,0], intersections[:,1], c=values, cmap=CMAP_NAME, s=5)

# scatter the (p, Φ) values
plt.subplot(325)
plt.contour(XY_GT[:,:,0], XY_GT[:,:,1], EMISS) 
plt.plot(FW[:,0], FW[:,1], 'w', lw=2, alpha=0.3)
plt.scatter(RM+ps*np.cos(Φs), ZM+ps*np.sin(Φs), c=SXR, cmap=CMAP_NAME, s=15)
plt.axis('equal')
plt.xlim(R0, R1); plt.ylim(Z0, Z1)
# plt.grid(False)

plt.subplot(326)
# simply plot the ps values (stem plot on the Φs)
plt.stem(Φs, ps)
plt.xlabel('Φ')
plt.ylabel('p')


plt.show()

### Bessel coefficients calculation
$f_{ml} (p) = a_{ml} \int_p^1 2  \frac{R_{ml}(r) cos(m * cos^{-1}(p/r)) r}{\sqrt{r^2 - p^2}} dr$

with $R_{ml}(r) = J_m(x_{ml}r)$

Converting in matrix form:

$\mathbf{f} = \mathbf{W} \cdot \mathbf{g} \\ \mathbf{g} = \mathbf{W}^{-1} \cdot \mathbf{f}$


Alternative (maybe this is more correct):

$f(p, \phi) = \sum_{m=0}^{\infty} f_m(p) e^{im\phi} $

In [10]:
# first rescale the ps by MINR
ps = ps/MINR

In [11]:
# recalculate the sxrs
def λfmlp(m,l,p): return lambda r: 2 * Rml(m, l, r) * Tm(m, p/r) * r / np.sqrt(r**2 - p**2) 
def calc_fml(m,l,p): return quad(λfmlp(m,l,p), p, 1)[0] # calculate the fmlp coefficients

# v2
# def λfmlp2(m,l,p): return lambda θ: np.cos(m*θ) * np.sin(J0(m, L)[l] * np.cos(θ-p)) 
def λfmlp2(m,l,p): return lambda θ: np.cos(m*θ) * np.sin(JZ[m,l] * np.cos(θ-p)) 
# def calc_fml2(m,l,p): return -2 * JP(J0(m,l)[l]) * quad(λfmlp2(m,l,p), 0, np.arccos(p))[0] # calculate the fmlp coefficients
def calc_fml2(m,l,p): return -2 * JP(m, JZ[m,l]) * quad(λfmlp2(m,l,p), 0, np.arccos(p))[0] # calculate the fmlp coefficients


In [ ]:
# recalculate the sxrs
def λfmlp(m,l,p): return lambda r: 2 * Rml(m, l, r) * Tm(m, p/r) * r / np.sqrt(r**2 - p**2) 
def calc_fml(m,l,p): return quad(λfmlp(m,l,p), p, 1)[0] # calculate the fmlp coefficients

# v2 (not correct)
# def λfmlp2(m,l,p): return lambda θ: np.cos(m*θ) * np.sin(J0(m, L)[l] * np.cos(θ-p)) 
def λfmlp2(m,l,p): return lambda θ: np.cos(m*θ) * np.sin(JZ[m,l] * np.cos(θ)-p) 
# def calc_fml2(m,l,p): return -2 * JP(J0(m,l)[l]) * quad(λfmlp2(m,l,p), 0, np.arccos(p))[0] # calculate the fmlp coefficients
def calc_fml2(m,l,p): return -2 * JP(m, JZ[m,l]) * quad(λfmlp2(m,l,p), 0, np.arccos(p))[0] # calculate the fmlp coefficients

# v3 (not correct)
def calc_fml3(m,l,p):
    def δn(n): return .5 if n == 0 else 1

    fml = 0
    Ns = [n for n in range(100) if n != m]
    for n in Ns:
        fml += δn(n) * J(n, JZ[m,l]) * np.sin(n*π/2) * (Um(m+n-1,p)/(m+n)) * (Um(m-n-1,p)/(m-n))
    return fml * (-2 * np.sqrt(1-p**2) * JP(m, JZ[m,l]))


f_pΦs = np.zeros((len(SXR)))
for i in tqdm(range(len(SXR))):
    f_pΦs[i] = 0
    for m in range(M):
        fm = 0
        for l in range(L):
            fml = calc_fml(m,l,ps[i])
            # fml2 = calc_fml2(m,l,ps[i])
            # fml3 = calc_fml3(m,l,ps[i])
            # assert np.isclose(fml, fml2), f'fml: {fml}, fml2: {fml2}'
            fm += A[m,l] * fml
        f_pΦs[i] += fm * np.exp(-1j*m*Φs[i])



print(f'f_pΦs: \n{f_pΦs}')
print(f'SXR: \n{SXR}')

# plot
plt.figure(figsize=(12,2))
plt.subplot(121)
plt.plot(f_pΦs, 'r', label='f_pΦs')
plt.plot(SXR, label='SXR')
plt.legend()
plt.grid(True)
plt.subplot(122)
plt.plot(f_pΦs, 'r', label='f_pΦs')
plt.legend()
plt.grid(True)
plt.show()

## Inverse problem